In [1]:
import numpy,sys,time
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import string
import gensim
from gensim.models import Word2Vec

Using TensorFlow backend.


In [2]:
def read_input(input_file):
    with open(input_file, 'rb') as f:
        for i, line in enumerate(f):
            yield gensim.utils.simple_preprocess(line)

In [3]:
doc = read_input('data.tsv')
# build vocabulary and train model
doc_list = [d for d in doc]
model = gensim.models.Word2Vec(
        doc_list,
        size=150,
        window=10,
        min_count=1,
        workers=10)
model.train(doc_list, total_examples=len(doc_list), epochs=10)

(34850946, 50344940)

In [4]:
w1 = 'something'
model.wv.most_similar(positive=w1)

/datasets/home/56/656/haa001/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('anything', 0.7059581279754639),
 ('things', 0.5725936889648438),
 ('nothing', 0.5648808479309082),
 ('whatever', 0.535612940788269),
 ('everything', 0.5327731370925903),
 ('it', 0.5194367170333862),
 ('what', 0.5147068500518799),
 ('someone', 0.5136159062385559),
 ('stuff', 0.4700457453727722),
 ('intra', 0.4530140161514282)]

In [5]:
model.wv['computers']

array([-1.2865996 ,  0.05312096, -0.15904526, -0.36874214,  0.93098485,
       -0.41446996,  0.9292068 , -0.5055322 , -0.49235085, -0.85127765,
        1.0466743 ,  0.55201286,  0.48503098,  1.1396943 ,  0.47737584,
        0.16407704,  0.08244048,  0.23501381,  0.41852632,  0.568707  ,
        0.9552854 , -0.46423876,  0.10247459,  0.27861083,  1.2197453 ,
        0.35090977, -0.49689606,  0.31761396, -0.07180959,  0.6022233 ,
       -0.8020828 ,  0.11679435,  0.18990673,  0.6967766 , -1.5653837 ,
        0.45386153,  0.20344369,  0.46711898, -0.642616  ,  0.3891348 ,
       -0.3411015 ,  0.82972777, -0.09639759, -0.45927635, -0.7186664 ,
       -0.06674694,  0.16246893, -0.51936454, -1.1227741 ,  0.30203584,
       -0.6738462 ,  0.44886115, -0.08223097, -0.00431547, -0.6888116 ,
        0.3094585 , -0.02324419,  0.22051877,  0.38138956,  0.58008736,
        0.6846353 ,  0.78361344, -0.41444343,  0.38082018,  0.6819345 ,
        0.90999985, -0.39837572, -0.75698006, -1.382539  ,  0.53

In [6]:
your_word_vector = model.wv['men']
model.wv.most_similar(positive=[your_word_vector], topn=5)

/datasets/home/56/656/haa001/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('men', 1.0),
 ('women', 0.7996717691421509),
 ('girls', 0.6573750972747803),
 ('males', 0.6254711151123047),
 ('females', 0.6128126978874207)]

In [7]:
seq_length = 5
dataX = []
dataY = []
for haiku_list in doc_list:
    for i in range(0, len(haiku_list)-1-seq_length):
        seq_in=haiku_list[i:i+seq_length]
        seq_out=haiku_list[i+seq_length]
        dataX.append([model.wv[w] for w in seq_in])
        dataY.append(model.wv[seq_out])

In [41]:
for v in dataX[0]:
    print(model.wv.most_similar(positive=[v], topn=5))

[('can', 0.9999999403953552), ('cant', 0.7830692529678345), ('cannot', 0.7410323023796082), ('could', 0.7139013409614563), ('will', 0.6706205010414124)]
[('you', 0.9999999403953552), ('they', 0.6642903089523315), ('we', 0.57551109790802), ('yourself', 0.5415840744972229), ('others', 0.4377296566963196)]
[('see', 1.0), ('hear', 0.5801050662994385), ('imagine', 0.5390194058418274), ('know', 0.5318241119384766), ('tell', 0.5078991055488586)]
[('how', 1.0), ('what', 0.5031564831733704), ('where', 0.4781579375267029), ('why', 0.4342767000198364), ('so', 0.4271538555622101)]
[('much', 0.9999999403953552), ('far', 0.5101966857910156), ('deeply', 0.4690285325050354), ('brisk', 0.4514276385307312), ('frequently', 0.4510418772697449)]


/datasets/home/56/656/haa001/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
# filename = 'data.tsv'
# raw_text=open(filename,'r').read().lower()

In [ ]:
# raw_text = raw_text.replace('<br>', ' ')
# processed_text = raw_text.replace('\n', ' ')
# raw_text = ''.join([c for c in raw_text if not c in string.punctuation])
# processed_text = ''.join([c for c in processed_text if not c in string.punctuation])
# words=sorted(list(set(processed_text.split(' '))))

In [ ]:
# word_to_int = dict((c,i) for i, c in enumerate(words))

In [ ]:
# n_words = len(processed_text.split(' '))
# n_vocab = len(words)

In [ ]:
# seq_length = 5
# lines = raw_text.split('\n')
# dataX = []
# dataY = []
# for haiku in lines:
#     haiku_list = haiku.split(' ')
#     haiku_list = [h for h in haiku_list if h != '']
#     for i in range(0, len(haiku_list)-1-seq_length):
#         seq_in=haiku_list[i:i+seq_length]
#         seq_out=haiku_list[i+seq_length]
#         dataX.append([word_to_int[w] for w in seq_in])
#         dataY.append(word_to_int[seq_out])

In [ ]:
n_patterns = len(dataX)
X = numpy.reshape(dataX, (n_patterns, seq_length, len(dataX[0][0])))
# X = X/float(n_vocab)
y = np_utils.to_categorical(dataY)
# y = dataY

In [58]:
model = Sequential()
# model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]),return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256,input_shape=(X.shape[1], X.shape[2]),return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256,input_shape=(X.shape[1], X.shape[2]),return_sequences=True))
# model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))
model.add(Dense(len(y), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# y = np_utils.to_categorical(dataY)
# X = dataX
model.fit(X, y, epochs=1, batch_size=200)

In [ ]:
# int_to_word=dict((i,c) for i,c in enumerate(words))

In [ ]:
numpy.random.seed(int(time.time()))
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print('pattern is', pattern)

sys.stdout.write("Generated Haiku: ")
generated=' '.join([int_to_word[value] for value in pattern])

syllables_count = 0

for i in range(10):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = ' ' + int_to_word[index] 
    #seq_in = [int_to_word[value] for value in pattern]
    #sys.stdout.write(result)
    generated+=result
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
sys.stdout.write("%s\n"%generated)